In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import warnings

matplotlib.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

In [2]:
# 企业信息
df1 = pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx', sheetname = 0) 
df1['企业代号'] = df1['企业代号'].apply(lambda x: int(x[1:]))

In [3]:
# 进项发票信息
df2 = pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx', sheetname = 2) 
df2['企业代号'] = df2['企业代号'].apply(lambda x: int(x[1:]))

In [4]:
# 销项发票信息
df3 = pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx', sheetname = 1)
df3['企业代号'] = df3['企业代号'].apply(lambda x: int(x[1:]))

In [5]:
df2 = df2[df2['发票状态'] == '有效发票']
df3 = df3[df3['发票状态'] == '有效发票']

In [6]:
df4 = pd.concat([df2, df3], axis = 0)
df4 = df4.reset_index()

In [7]:
# 计算现金流稳定程度
current_std = []
for i in range(124, 426):
    current_std.append(df4[df4['企业代号'] == i]['价税合计'].std())

df1['现金流稳定程度'] = current_std

In [8]:
df1.head()

,企业代号,企业名称,现金流稳定程度
0,124,个体经营E124,241550.260104
1,125,个体经营E125,244592.838114
2,126,个体经营E126,401430.943412
3,127,个体经营E127,219295.616154
4,128,个体经营E128,181211.712302


In [9]:
# 计算供应商密切程度
df2_up_invoice = df2.groupby(['企业代号','销方单位代号'])[['发票号码']].count().reset_index()
df2_invoice_std = []

for i in range(124, 426):
    df2_invoice_std.append(df2_up_invoice[df2_up_invoice['企业代号'] == i]['发票号码'].std())

In [10]:
# 计算客户粘性
df3_down_invoice = df3.groupby(['企业代号','购方单位代号'])[['发票号码']].count().reset_index()
df3_invoice_std = []

for i in range(124, 426):
    df3_invoice_std.append(df3_down_invoice[df3_down_invoice['企业代号'] == i]['发票号码'].std())

In [11]:
df1['供应商密切程度'] = df2_invoice_std
df1['客户粘性'] = df3_invoice_std

In [12]:
from numpy import nan as NaN
df1['供应商密切程度'] = df1['供应商密切程度'].fillna(df1['供应商密切程度'].max())
df1['客户粘性'] = df1['客户粘性'].fillna(df1['客户粘性'].max())

In [13]:
df1.head()

,企业代号,企业名称,现金流稳定程度,供应商密切程度,客户粘性
0,124,个体经营E124,241550.260104,45.675909,32.612417
1,125,个体经营E125,244592.838114,41.951639,35.089615
2,126,个体经营E126,401430.943412,17.306402,18.093517
3,127,个体经营E127,219295.616154,20.008484,212.511310
4,128,个体经营E128,181211.712302,28.600886,417.315400


In [14]:
df1['行业'] = pd.read_excel('附件2企业的行业分类.xlsx')['行业分类']

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 6 columns):
企业代号       302 non-null int64
企业名称       302 non-null object
现金流稳定程度    302 non-null float64
供应商密切程度    302 non-null float64
客户粘性       302 non-null float64
行业         302 non-null object
dtypes: float64(3), int64(1), object(2)
memory usage: 14.2+ KB


In [16]:
pd.set_option('display.max_columns', None) # 显示所有列
pd.set_option('display.max_rows', None) # 显示所有行

In [17]:
df1.groupby('行业')[['现金流稳定程度','供应商密切程度','客户粘性']].describe()

现金流稳定程度                                               \
                   count           mean            std            min   
行业                                                                      
个体                  56.0   94696.242549  100902.945359    8191.268464   
交通运输、仓储和邮政业         11.0   62427.597117   45681.604762   14992.152308   
住宿和餐饮业               2.0    2253.178163     487.080738    1908.760070   
信息传输、软件和信息技术服务业     32.0   49651.579447   38307.559521    4303.532423   
农、林、牧、渔业             3.0   40291.862117   10878.108075   27958.802991   
制造业                 37.0   40834.965723   32686.928487    2325.834092   
卫生和社会工作             12.0   59824.193689  106886.571596    4178.121743   
居民服务、修理和其他服务业       12.0   61572.367231   68838.403958    4272.657115   
建筑业                 58.0   94689.553472   94706.040733    2449.182768   
房地产业                 4.0   25391.727807   17681.348031    3619.756230   
批发和零售业              37.0   56489.052074   94873.237036    1529.497783   
教育                   1.0   26289.343924            NaN   26289.343924   
文化、体育和娱乐业           21.0   27801.153459   18451.688755     918.158415   
水利、环境和公共设施管理业        6.0   36196.492846   21884.250459   12072.819525   
电力、热力、燃气及水生产和供应业     3.0  111555.787172   12846.463553   99155.902988   
科学研究和技术服务业           1.0   14621.944821            NaN   14621.944821   
租赁和商务服务业             4.0   27975.885772   29520.436090    2257.331508   
采矿业                  1.0   57072.203393            NaN   57072.203393   
金融业                  1.0  261868.813996            NaN  261868.813996   

                                                                              \
                            25%            50%            75%            max   
行业                                                                             
个体                 39467.404917   51220.320506  102749.032885  426676.063554   
交通运输、仓储和邮政业        32076.495919   48532.465074   85222.045432  159283.317903   
住宿和餐饮业              2080.969117    2253.178163    2425.387210    2597.596256   
信息传输、软件和信息技术服务业    27827.177596   40341.114259   58104.780082  147837.925015   
农、林、牧、渔业           36177.217829   44395.632666   46458.391680   48521.150693   
制造业                17523.634147   38719.543025   48412.326610  156058.288952   
卫生和社会工作            16482.541498   22093.013884   49305.206931  392909.463254   
居民服务、修理和其他服务业      16634.267793   46307.947471   56723.543456  223323.590618   
建筑业                36405.792649   57083.079419  117998.011188  450160.531747   
房地产业               19276.704854   25534.427663   31649.450616   46878.299671   
批发和零售业              4541.746805   32404.547604   58210.822449  511353.893289   
教育                 26289.343924   26289.343924   26289.343924   26289.343924   
文化、体育和娱乐业          12803.237242   28451.967506   40474.940329   67092.254599   
水利、环境和公共设施管理业      17207.438818   35656.956914   52442.979754   64696.303571   
电力、热力、燃气及水生产和供应业  104930.416149  110704.929309  117755.729264  124806.529218   
科学研究和技术服务业         14621.944821   14621.944821   14621.944821   14621.944821   
租赁和商务服务业           14496.989504   19565.969210   33044.865477   70514.273159   
采矿业                57072.203393   57072.203393   57072.203393   57072.203393   
金融业               261868.813996  261868.813996  261868.813996  261868.813996   

                 供应商密切程度                                               \
                   count       mean         std        min        25%   
行业                                                                      
个体                  56.0  16.508559   16.090850   0.000000   8.123888   
交通运输、仓储和邮政业         11.0  22.458130   33.407575   0.577350   6.791637   
住宿和餐饮业               2.0  12.701205    6.547015   8.071766  10.386486   
信息传输、软件和信息技术服务业     32.0  13.230044    9.815300   0.547723   6.576348   
农、林、牧、渔业             3.0   3.737200    5.215869   0.500000   0.728714   
制造业                 37.0  18.545132   50.722347   2.147018   5.51

In [19]:
df1.to_excel('data3.xlsx')